In [15]:
import napari
import h5py
from pathlib import Path
from lca.ndt.segmentation import segment_nuclei_cellpose_2DT
from lca.ndt.LapTracker import LapTracker
from lca.ndt.measure import measure_intensity_2DT, measure_morphology_2DT

# Object segmentation with cellpose

In this notebook, we will try to segment cells/nuclei in microscopy images. We will use a deep-learning based approach to segment the objects in this course (as opposed to more classical approaches). The largest disadvantage of deep-learning based approaches is that they often require training data that is hard to produce. In this course, we will circumvent this by using cellpose (http://www.cellpose.org/). Cellpose is a generalist deep-learning based approach that performs very well out-of-the-box to segment nuclei and cytoplasm of cells in images. 


### Nuclei segmentation
Let's start by loading some of the images that you have produced into memory. Make sure to adjust the path!:

In [16]:
img_path = Path(r'X:\20211028_Bio325_Sec16\hdf5')
site = 7
file = h5py.File(img_path.joinpath('site_%04d.hdf5' % site), "r")

Now let's look at the data in napari. Cellpose has a plugin for napari that let's you use a graphical user interface to do segmentation!

In [7]:
nuclei_img = file['intensity_images/sdcDAPIxmRFPm'][0,:,:]

viewer = napari.Viewer()
viewer.add_image(nuclei_img)

creating new log file
2021-10-29 14:53:30,952 [INFO] WRITING LOG OUTPUT TO C:\Users\Adrian\.cellpose\run.log


<Image layer 'nuclei_img' at 0x29ad88f75b0>

2021-10-29 14:53:34,210 [ERROR] Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\jupyter_client\threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\asyncio\base_events.py", line 618, in run_until_complete
    self._check_running()
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\asyncio\base_events.py", line 580, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
2021-10-29 14:53:34,210 [ERROR] Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\zmq\

In the IT introduction we already used a plugin in napari. You can open plugins from the menu bar under Plugins. There you should see "cellpose-napari: cellpose".

Try to play around with the settings to get a decent segmentation of the nuclei. 
IMPORTANT: keep the "resample dynamics" box unticked. If you use this option it will take a very long time to generate the segmentation.

### Cell segmentation

If you are happy with your nuclei segmentation, try to load an image of the membrane marker and see if you can generate a segmentation of the cells as well!

In [8]:
# try cell segmentation
cell_img = file['intensity_images/sdcCy5'][0,:,:]

viewer = napari.Viewer()
viewer.add_image(cell_img)

C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\abc.py:123: RuntimeWarning: coroutine 'get_msg' was never awaited
  return _abc_subclasscheck(cls, subclass)


<Image layer 'cell_img' at 0x29afea2b670>

2021-10-29 14:53:46,624 [ERROR] Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\jupyter_client\threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\asyncio\base_events.py", line 618, in run_until_complete
    self._check_running()
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\asyncio\base_events.py", line 580, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
2021-10-29 14:53:46,624 [ERROR] Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\zmq\

# Segmentation and object tracking with 2DT data 
Since the data you have generated are movies, we can think of them as 3D arrays in which the third dimension is time (2DT). We can also do segmentations for a 2DT array directly instead of only a single image. Let's start by loading a 3D image stack. This stack will have the dimensions (t, y, x). Run the next cell and look at the images in napari.

In [10]:
nuclei_img_2DT = file['intensity_images/sdcDAPIxmRFPm'][0:5,:,:]
viewer = napari.Viewer()
viewer.add_image(nuclei_img_2DT)

<Image layer 'nuclei_img_2DT' at 0x29a9e131b50>

2021-10-29 14:58:37,599 [ERROR] Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\jupyter_client\threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\asyncio\base_events.py", line 618, in run_until_complete
    self._check_running()
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\asyncio\base_events.py", line 580, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
2021-10-29 14:58:37,614 [ERROR] Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "C:\Users\Adrian\miniconda3\envs\bio325_2021\lib\site-packages\zmq\

The next cell will create cellpose segmentations for the nuclei at each timepoint of the 3D image and add it to napari as labels. It is doing the same thing as the cellpose plugin for napari, just without a graphical user interface. Make sure that you adapt the diameter with the diameter that worked for you during testing with the cellpose plugin for napari

In [13]:
nuclei_2DT = segment_nuclei_cellpose_2DT(nuclei_img_2DT, diameter=150, resample=False)
viewer.add_labels(nuclei_2DT)

2021-10-29 14:59:13,551 [INFO] >>>> using CPU
2021-10-29 14:59:19,609 [INFO] ~~~ FINDING MASKS ~~~
2021-10-29 14:59:28,041 [INFO] >>>> TOTAL TIME 8.43 sec
2021-10-29 14:59:28,182 [INFO] >>>> using CPU
2021-10-29 14:59:34,037 [INFO] ~~~ FINDING MASKS ~~~
2021-10-29 14:59:42,478 [INFO] >>>> TOTAL TIME 8.44 sec
2021-10-29 14:59:42,631 [INFO] >>>> using CPU
2021-10-29 14:59:48,486 [INFO] ~~~ FINDING MASKS ~~~
2021-10-29 14:59:56,972 [INFO] >>>> TOTAL TIME 8.49 sec
2021-10-29 14:59:57,115 [INFO] >>>> using CPU
2021-10-29 15:00:02,947 [INFO] ~~~ FINDING MASKS ~~~
2021-10-29 15:00:11,433 [INFO] >>>> TOTAL TIME 8.49 sec
2021-10-29 15:00:11,566 [INFO] >>>> using CPU
2021-10-29 15:00:17,420 [INFO] ~~~ FINDING MASKS ~~~
2021-10-29 15:00:25,773 [INFO] >>>> TOTAL TIME 8.35 sec


<Labels layer 'nuclei_2DT' at 0x29ac694e9a0>

As you can see we now have labels for each timepoint. As humans, it is easy for us to see which labels correspond to the same nucleus in the image over time. But the computer doesn't have a clue so far, so we need to tell it somehow which labels belong together over time. This process is called "object tracking". There is a lot of object tracking algorithms out there. Today we will use a so called "linear assignment problem" tracker that is based on the following publication: https://www.nature.com/articles/nmeth.1237. 

The following cell will perform the tracking for you. As you know by now, napari has multiple layer "types" (e.g. image layers and label layers). There is also a layer type called tracks layer. This is a very handy way to visualize tracking results. The results of the tracking will be added to your napari viewer on a tracks layer. Run the following cell and see what happens on the napari viewer. Are you happy with the tracking? If not, try to play with the parameters of the tracker (max_distance, time_window, max_split_distance, max_gap_closing_distance) and run the cell again.

In [14]:
tracker = LapTracker(max_distance=50, time_window=3, max_split_distance=100, max_gap_closing_distance=100)
tracked_labels, tracks = tracker.track_label_images(nuclei_2DT)
viewer.add_tracks(tracks[['track_id', 'timepoint', 'centroid-0', 'centroid-1']])

measuring centroids |################################| 5/5
linking objects across time |################################| 4/4


linking track segments across timepoints
calculating average displacement per segment


switching labels |################################| 4/4


<Tracks layer 'Tracks' at 0x29ac695e190>

# Feature measurements

Now that we have segmentations we can start to think about things we want to measure. Typical measurements that are interesting are, for example, morphology features (how do object shapes look like?) and intensity features (what kind of intensities do we observe inside of objects).

We can measure these features for 2DT stacks with the measure_morphology_2DT and measure_intensity_2DT functions that we imported at the beginning of the file. Let's see what their output looks like

In [17]:
# measure morphology  
morphology_features = measure_morphology_2DT(nuclei_2DT)

In [18]:
morphology_features.head()

,label,area,bbox-0,bbox-1,bbox-2,bbox-3,bbox_area,centroid-0,centroid-1,convex_area,...,moments_normalized-2-2,moments_normalized-2-3,moments_normalized-3-0,moments_normalized-3-1,moments_normalized-3-2,moments_normalized-3-3,orientation,perimeter,solidity,timepoint
0,1,4312,0,27,80,98,5680,40.412106,63.559137,4727,...,0.004355,0.000028,-0.002428,-0.001798,0.000137,-0.000168,-0.430977,292.727922,0.912206,0
1,2,3214,0,168,53,230,3286,25.495955,199.093653,3246,...,0.006575,0.000184,0.000445,0.000716,-0.000157,0.000167,1.413016,225.414214,0.990142,0
2,3,4187,0,398,71,468,4970,33.284691,434.917124,4490,...,0.005710,0.000082,0.000589,-0.001195,-0.000500,-0.000208,-0.452212,275.071068,0.932517,0
3,4,5852,0,751,98,830,7742,49.994874,789.943096,6381,...,0.004910,-0.000126,-0.003219,0.004338,-0.000233,0.000414,0.359201,344.142136,0.917098,0
4,5,5150,0,857,71,945,6248,31.632427,901.240971,5515,...,0.005960,0.000449,0.003871,0.000207,-0.000511,-0.000041,1.528047,310.485281,0.933817,0


In [19]:
# measure intensities
intensity_features = measure_intensity_2DT(nuclei_2DT, nuclei_img_2DT)

In [20]:
intensity_features.head()

,label,max_intensity,mean_intensity,min_intensity,timepoint
0,1,1010,623.099258,130,0
1,2,1113,597.633790,128,0
2,3,2139,942.042274,183,0
3,4,1800,1219.423958,124,0
4,5,2147,1214.526990,104,0
